# 通过 LiteLLM 定制 llm 和嵌入模型

https://github.com/truera/trulens/blob/main/trulens_eval/examples/expositional/models/litellm_quickstart.ipynb

初步结论：

- 对openai的依赖很高，包括嵌入模型
- 国内模型都缺少token计费，无法作为评估模型
- trulen有很多隐藏的bug，调试起来问题较多

In [1]:
%%time

import os

base_url = "http://ape:3000/v1"
api_key = "sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750"

os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_API_BASE"] = base_url

CPU times: user 8 µs, sys: 2 µs, total: 10 µs
Wall time: 11.4 µs


In [2]:
%%time

import nest_asyncio
nest_asyncio.apply()

CPU times: user 512 µs, sys: 96 µs, total: 608 µs
Wall time: 453 µs


In [3]:
university_info = """
The University of Washington, founded in 1861 in Seattle, is a public research university
with over 45,000 students across three campuses in Seattle, Tacoma, and Bothell.
As the flagship institution of the six public universities in Washington state,
UW encompasses over 500 buildings and 20 million square feet of space,
including one of the largest library systems in the world.
"""

In [4]:
%%time

from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

embedding_function = OpenAIEmbeddingFunction(
    # openai embedding
    api_key=api_key,
    model_name="text-embedding-ada-002",
    api_base=base_url,

    # # bge，可以，但是trulens中会报错
    # api_key="ollama",
    # model_name="quentinz/bge-large-zh-v1.5",
    # api_base="http://monkey:11434/v1",

    # # gte
    # api_key="ollama",
    # model_name="rjmalagon/gte-qwen2-1.5b-instruct-embed-f16",
    # api_base="http://monkey:11434/v1",
)

CPU times: user 894 ms, sys: 180 ms, total: 1.07 s
Wall time: 882 ms


In [5]:
import chromadb

chroma_client = chromadb.Client()
vector_store = chroma_client.get_or_create_collection(
    name="Universities",
    embedding_function=embedding_function
)

In [6]:
%%time

vector_store.add(
    "uni_info",
    documents=university_info,
)

CPU times: user 67.1 ms, sys: 568 µs, total: 67.6 ms
Wall time: 1.08 s


In [7]:
%%time

from trulens_eval import Tru
from trulens_eval.tru_custom_app import instrument

tru = Tru()
tru.reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.
CPU times: user 5.92 s, sys: 412 ms, total: 6.34 s
Wall time: 6.18 s


In [8]:
%%time

import litellm
from litellm import embedding


class RAG_from_scratch:
    @instrument
    def retrieve(self, query: str) -> list:
        """
        Retrieve relevant text from vector store.
        """
        results = vector_store.query(
            query_embeddings=embedding(
                # openai
                model="text-embedding-ada-002", 
                input=query

                # # bge，litellm.APIError: APIError: OpenAIException - 'NoneType' object has no attribute 'get'
                # model = "openai/quentinz/bge-large-zh-v1.5",
                # api_key="ollama",
                # api_base="http://monkey:11434/v1",
                # input=query,

                # # gte, litellm.APIError: APIError: OpenAIException - 'NoneType' object has no attribute 'get'
                # model = "openai/rjmalagon/gte-qwen2-1.5b-instruct-embed-f16",
                # api_key="ollama",
                # api_base="http://monkey:11434/v1",
                # input=query,
                
            ).data[0]["embedding"],
            n_results=2,
        )
        return results["documents"]

    @instrument
    def generate_completion(self, query: str, context_str: list) -> str:
        """
        Generate answer from context.
        """
        completion = (
            litellm.completion(
                # model="gpt-3.5-turbo",
                model="ollama/qwen2",
                api_base="http://monkey:11434",
                temperature=0,
                messages=[
                    {
                        "role": "user",
                        "content": f"We have provided context information below. \n"
                        f"---------------------\n"
                        f"{context_str}"
                        f"\n---------------------\n"
                        f"Given this information, please answer the question: {query}",
                    }
                ],
            )
            .choices[0]
            .message.content
        )
        return completion

    @instrument
    def query(self, query: str) -> str:
        context_str = self.retrieve(query)
        completion = self.generate_completion(query, context_str)
        return completion


rag = RAG_from_scratch()

CPU times: user 41 µs, sys: 4 µs, total: 45 µs
Wall time: 47.4 µs


In [9]:
%%time

import numpy as np

from trulens_eval import Feedback
from trulens_eval import LiteLLM
from trulens_eval import Select

# Initialize LiteLLM-based feedback function collection class:
provider = LiteLLM(
    model_engine="gpt-3.5-turbo",

    # # 不行: NotFoundError: litellm.NotFoundError: Model not in model_prices_and_context_window.json.
    # model_engine="ollama/qwen2",
    # api_base="http://monkey:11434",

    # model_engine="qwen-turbo",
)

# Define a groundedness feedback function
f_groundedness = (
    Feedback(
        provider.groundedness_measure_with_cot_reasons, name="Groundedness"
    )
    .on(Select.RecordCalls.retrieve.rets.collect())
    .on_output()
)

# Question/answer relevance between overall question and answer.
f_answer_relevance = (
    Feedback(provider.relevance_with_cot_reasons, name="Answer Relevance")
    .on(Select.RecordCalls.retrieve.args.query)
    .on_output()
)

# Question/statement relevance between question and each context chunk.
f_context_relevance = (
    Feedback(
        provider.context_relevance_with_cot_reasons, name="Context Relevance"
    )
    .on(Select.RecordCalls.retrieve.args.query)
    .on(Select.RecordCalls.retrieve.rets.collect())
    .aggregate(np.mean)
)

f_coherence = Feedback(
    provider.coherence_with_cot_reasons, name="coherence"
).on_output()

✅ In Groundedness, input source will be set to __record__.app.retrieve.rets.collect() .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Answer Relevance, input prompt will be set to __record__.app.retrieve.args.query .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input question will be set to __record__.app.retrieve.args.query .
✅ In Context Relevance, input context will be set to __record__.app.retrieve.rets.collect() .
✅ In coherence, input text will be set to __record__.main_output or `Select.RecordOutput` .
CPU times: user 82.5 ms, sys: 50 µs, total: 82.5 ms
Wall time: 82.1 ms


In [10]:
from trulens_eval import TruCustomApp

tru_rag = TruCustomApp(
    rag,
    app_id="RAG v1",
    feedbacks=[
        f_groundedness,
        f_answer_relevance,
        f_context_relevance,
        f_coherence,
    ],
)

In [11]:
import nltk
# [nltk_data] Error loading punkt: <urlopen error [Errno 111] Connection
nltk.set_proxy('http://myproxy:7890')

In [12]:
with tru_rag as recording:
    rag.query("Give me a long history of U Dub")

In [14]:
tru.get_leaderboard(app_ids=["RAG v1"])

,coherence,Groundedness,Context Relevance,Answer Relevance,latency,total_cost
app_id,,,,,,
RAG v1,0.9,0.268421,0.8,1.0,15.0,0.0
